# Secular dynamics with celmech

Import packages

In [ ]:

from celmech.disturbing_function import *
from sympy import symbols, init_printing,Function,simplify,collect,cos
init_printing()
import numpy as np
from celmech.poincare import Poincare, PoincareHamiltonian
import matplotlib
%matplotlib inline

## Use sympy to derive a 4th order Laplace expansion of the secular disturbing function:

In [ ]:
e1,e2,w1,w2=symbols("e1 e2 varpi1 varpi2")
sdf4 = secular_DF(e1,e2,w1,w2,4)

Print the symbolic expression with sympy

In [ ]:
sdf4

Substitute particular values to evaluate for a choice of semi-major axis ratio. Use the celmech routine 'laplace_B' to numerically evaluate Laplace coefficients:

In [ ]:
Nalpha = 0.4
sdf4.subs([(S('alpha'),Nalpha),(Function('b'),laplace_B)])

# Compare N-body with celmech 

## Set up a Rebound integration

In [ ]:
np.random.seed(123)
import rebound
def make_sim(pratio1,pratio2):
    mass = 3.e-5
    
    e1,e2,e3 = 0.2 * np.random.rand(3) 
    w1,w2,w3 = 2*np.pi*np.random.rand(3)
    
    sim = rebound.Simulation()
    sim.G = 4*np.pi**2

    sim.add(m=1.)
    sim.add(m=mass, e=e1, P=1.,pomega=w1)
    sim.add(m=mass, e=e2, P=pratio1, pomega=w2)
    sim.add(m=mass, e=e3, P=pratio1*pratio2, pomega=w3)
    
    sim.move_to_com()
    return sim     

sim=make_sim(np.sqrt(7),np.sqrt(4.3))
sim.integrator='whfast'
sim.dt=sim.particles[1].P / 25.

rebound.plotting.OrbitPlot(sim);

## Use Rebound simulation to initalize celmech system in Poincare variables

In [ ]:
pvars = Poincare.from_Simulation(sim)
Hsim = PoincareHamiltonian(pvars)

## Add secular disturbing function terms to 4th order

Secular terms are added for each pairwise interaction

In [ ]:
Nbodies = Hsim.state.N
expand_order=4
for i in range(1,Nbodies):
    for j in range(i+1,Nbodies):
        Hsim.add_secular_terms(i,j,expand_order)

Set secular mode to disable (Lambda,lambda) degrees of freedom in Hamiltonian

In [ ]:
Hsim.set_secular_mode()

## Run the integrations

In [ ]:
tFin,Nout= 4e5,500
times= np.linspace(0,tFin,Nout)
rebound_ecc = np.zeros((3,Nout))
celmech_ecc = np.zeros((3,Nout))
for i,t in enumerate(times):
    for j in range(1,sim.N):
        celmech_ecc[j-1,i] = Hsim.state.particles[j].eccentricity
        rebound_ecc[j-1,i] = sim.particles[j].e
    Hsim.integrate(t)
    sim.integrate(t)



## Compare results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig,ax = plt.subplots(1,figsize=(10,5))
for i,col in enumerate(['k','r','b']):
    ax.plot(times,rebound_ecc[i],'%s'%col,label='Nbody,planet %d'%(i+1))
    ax.plot(times,celmech_ecc[i],'%s-.'%col,label='celmech,planet %d'%(i+1))

ax.set_xlabel('Time [planet 1 orbits]')
ax.set_ylabel('Eccentricity')
ax.legend()
